# Question 1

Write your own code to fit a logistic regression model to the data set described below in a pro- gramming language of your choice. (**IMPORTANT: DO NOT USE ANY IN-BUILT LIBRARIES**)

#### Description of Data Set 1:

This data set describes the operating conditions of a reactor and contains class labels about whether the reactor will operate or fail under those operating conditions. Your job is to construct a logistic regression model to predict the same.

 - **q1_data_matrix.csv**: This file contains a 1000 × 5 data matrix. The 5 features are the operating conditions of the reactor; their corresponding ranges are described below:
 
    1. **Temperature:** 400-700 K
    2. **Pressure:** 1-50 bar
    3. **Feed Flow Rate:** 50-200 kmol/hr
    4. **Coolant Flow Rate:** 1000-3600 L/hr
    5. **Inlet Reactant Concentration:** 0.1-0.5 mol fraction
    
    
 - **q1_labels.csv**: This file contains a 1000 × 1 vector of 0/1 labels for whether the reactor will operate or fail under the corresponding operating conditions.
     <br><br>
     
     -  0: The reactor will operate well under the operating conditions
     -  1: The reactor fails under the operating conditions

#### Some General Guidelines:

1. Partition your data into a training set and a test set. Keep **70%** of your data for **training** and set aside the remaining **30%** for **testing.**
2. Fit a logistic regression model on the training set. Choose an appropriate objective function to quantify classification error. **Manually code for the gradient descent procedure** used to find optimum model parameters. (**Note:** You may need to perform multiple initializations to avoid local minima)
3. Evaluate the performance of above model on your test data. Report the **confusion matrix** and the F1 **Score**.

***

# Solution

### Import Libraries

In [1]:
import numpy as np
import pandas as pd

### Load data

Use `pandas.read_csv` method to load data and label the columns accordingly.

In [2]:
features = pd.read_csv('../data/q1_data_matrix.csv', header=None, names=['temp', 'press', 'ffr', 'cfr', 'irc'])
labels = pd.read_csv('../data/q1_labels.csv', header=None, names=['oper'])

### Data Exploration

Have a look on the first few rows using `pandas.DataFrame.head` method.

In [3]:
features.head()

,temp,press,ffr,cfr,irc
0,406.86,17.66,121.83,2109.20,0.1033
1,693.39,24.66,133.18,3138.96,0.3785
2,523.10,23.23,146.55,1058.24,0.4799
3,612.86,40.97,94.44,1325.12,0.3147
4,500.28,37.44,185.48,2474.51,0.2284


In [4]:
labels.head()

,oper
0,0.0
1,0.0
2,1.0
3,1.0
4,0.0


All features are real numbers so no encoding required.
<br>
Get the number of samples using `pandas.DataFrame.shape` method.

In [5]:
features.shape[0]

1000

As the number is just 1000 no need to use batch or stochastic methods for gradient descent.
<br>
Use `pandas.DataFrame.isnull` & `pandas.DataFrame.sum` together to get the count of rows with null values.

In [6]:
features.isnull().sum()

temp     0
press    0
ffr      0
cfr      0
irc      0
dtype: int64

There is no null values in the data. No need to drop any rows.
<br>
Find the distribution of samples across two classes using `pandas.Series.value_counts`.

In [7]:
labels["oper"].value_counts()

0.0    585
1.0    415
Name: oper, dtype: int64

Samples are almost equally distributed across classes.

In [8]:
pd.concat([features, labels["oper"]], axis=1).groupby("oper").mean()

,temp,press,ffr,cfr,irc
oper,,,,,
0.0,546.150291,24.906256,121.623419,2785.807744,0.303489
1.0,547.634964,26.320747,129.829783,1605.060819,0.301568


Average *Coolant Flow Rate* is very less for `oper=1` while every other parameter averages are almost same for both. 

### Split Data

Split data into train and test, 70% and 30% respectively using `numpy.split`.

In [9]:
[train_X, test_X] = np.split(features, [int(0.7*features.shape[0])], axis=0)
[train_Y, test_Y] = np.split(labels, [int(0.7*labels.shape[0])], axis=0)

Check distribution of samples across the classes in the training set.

In [10]:
train_Y["oper"].value_counts()

0.0    398
1.0    302
Name: oper, dtype: int64

### Feature Scaling

Subtract every feature by corresponding mean and divide by corresponding standard deviation. Use `pandas.DataFrame.mean` and `pandas.DataFrame.std` for mean and standard deviation respectively.

In [11]:
X = (train_X-train_X.mean())/train_X.std()
X.head()

,temp,press,ffr,cfr,irc
0,-1.598642,-0.534905,-0.063694,-0.207417,-1.705472
1,1.685504,-0.044677,0.200960,1.142378,0.689765
2,-0.266323,-0.144823,0.512715,-1.585001,1.572312
3,0.762487,1.097555,-0.702361,-1.235179,0.134473
4,-0.527881,0.850340,1.420466,0.271426,-0.616649


### Bias Term

Add a column of all ones to train_X.

In [12]:
X.insert(0, 'bias', 1)
X.head()

,bias,temp,press,ffr,cfr,irc
0,1,-1.598642,-0.534905,-0.063694,-0.207417,-1.705472
1,1,1.685504,-0.044677,0.200960,1.142378,0.689765
2,1,-0.266323,-0.144823,0.512715,-1.585001,1.572312
3,1,0.762487,1.097555,-0.702361,-1.235179,0.134473
4,1,-0.527881,0.850340,1.420466,0.271426,-0.616649


### Sigmoid 

Define a function `sigmoid` that computes sigmoid of input values.

In [13]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
sigmoid(0)

0.5

### Loss 

Define a function `cost` that computes loss given prediction and labels.

In [14]:
def cost(probability, label):
    return (-label*np.log(probability)-(1-label)*np.log(1-probability)).mean()

### Gradient

Define `gradient` function that computes gradient with respect to each parameter given features, labels and parameters.

In [15]:
def gradient(features, parameters, labels):
    h = sigmoid(np.dot(features, parameters))
    return np.dot(features.transpose(), h-labels)/labels.shape[0]

### Gradient Descent

`gradient_descent` function that performs iterations of gradient descent and returns parameters given features, labels, learning rate and number of iterations.

In [16]:
def gradient_descent(features, labels, learning_rate, number_of_iterations):
    parameters = np.random.normal(0, 1, features.shape[1])
    for i in range(number_of_iterations):
        grad = gradient(features, parameters, labels)
        parameters -= learning_rate*grad
        '''print(i+1, cost(sigmoid(np.dot(features, parameters)), labels), grad)'''
    return parameters

### Predict

`predict` function that returns the predicted labels given features, parameters and threshold.

In [17]:
def predict(features, parameters, threshold):
    return sigmoid(np.dot(features, parameters)) >= threshold

### Misclassification Error

`misclassification_error` function returns misclassification error.

In [18]:
def misclassification_error(predictions, labels):
    return (predictions!=labels).mean()

### Driver Loop

Run `gradient_descent` and print cost and gradient at each iterations 

In [19]:
Y = train_Y.iloc[:, 0]
theta = gradient_descent(X, Y, 0.01, 1500)
print(misclassification_error(predict(X, theta, 0.5), Y))
X_t = (test_X-test_X.mean())/test_X.std()
X_t.insert(0, 'bias', 1)
Y_t = test_Y.iloc[:, 0]
print(misclassification_error(predict(X_t, theta, 0.5), Y_t))

0.06428571428571428
0.1


Perform iterations of `gradient_descent` with multiple initialization and print misclassification error.

In [20]:
for i in range(20):
    theta_i = gradient_descent(X, Y, 0.01, 1500)
    mc_error_train = misclassification_error(predict(X, theta_i, 0.5), Y)
    mc_error_test = misclassification_error(predict(X_t, theta_i, 0.5), Y_t)
    print(i+1, round(mc_error_train, 4), round(mc_error_test, 4), theta_i)

1 0.0614 0.1067 [-0.48669393  0.00700717  0.42314132  0.17960628 -2.20864394  0.20135226]
2 0.0743 0.1133 [-0.35615802  0.08905102  0.30049108  0.35165182 -1.98849167  0.01629144]
3 0.0657 0.1267 [-0.36246268  0.16738791  0.42733766  0.19111541 -1.89718749 -0.02118091]
4 0.0543 0.09 [-0.56482636  0.10096672  0.31344391  0.4071495  -2.19584181  0.12585575]
5 0.0743 0.1133 [-0.45949211  0.2087388   0.32651188  0.46893444 -2.26295203  0.17071384]
6 0.0643 0.1 [-5.19511783e-01 -5.65543215e-05  3.16389908e-01  3.60090242e-01
 -2.16593152e+00 -4.68847475e-02]
7 0.0543 0.1033 [-0.45871588  0.15515207  0.33834754  0.2379893  -1.92682627  0.02405955]
8 0.0629 0.1167 [-0.44426825  0.23178596  0.40359731  0.29411205 -1.96157201  0.02820587]
9 0.0943 0.1333 [-0.23623519 -0.03492256  0.41872851  0.35162414 -2.03271458 -0.04669111]
10 0.0671 0.0967 [-0.76798016  0.33876889  0.37694651  0.46298355 -2.46983007 -0.17594117]
11 0.0843 0.1267 [-0.33044129 -0.08065153  0.25199863  0.15918316 -1.97258216  

In [21]:
theta = np.array([-0.6088456, 0.08545795, 0.20969599, 0.45275746, -2.05835641, 0.09583985])

### Confusion Matrix

`confusion_matrix` function which returns confusion matrix given predicted labels and original labels.

In [ ]:
def confusion_matrix(predictions, labels):
    true_positives = np.logical_and(predictions, labels).sum()
    false_positives = np.logical_and(predictions, np.logical_not(labels)).sum()
    false_negatives = np.logical_and(np.logical_not(predictions), labels).sum()
    true_negatives = np.logical_not(np.logical_or(predictions, labels)).sum()
    return pd.DataFrame(data={'Predicted 0': [true_negatives, false_negatives], 'Predicted 1': [false_positives, true_positives]}, index=['Actual 0', 'Actual 1'])

X = (features-features.mean())/features.std()
X.insert(0, 'bias', 1)
Y = labels.iloc[:, 0]
CF = confusion_matrix(predict(X, theta, 0.5), Y)
CF

### F1 Score

`f1_score` function calculates F1 score given confusion matrix.

In [ ]:
def f1_score(confusion_matrix):
    return (2*confusion_matrix["Predicted 1"]["Actual 1"])/(2*confusion_matrix["Predicted 1"]["Actual 1"]+confusion_matrix["Predicted 0"]["Actual 1"]+confusion_matrix["Predicted 1"]["Actual 0"])

f1_score(CF)

# Question 2

Use the same code developed in Question 1 to fit a logistic regression model to the dataset described below.

#### Description of Data Set 2:

This data set contains data for credit card fraud detection.

-  **q2_data_matrix.csv:** This file contains a 100 × 5 data matrix. The 5 features and their corresponding ranges are described below:
 
     1. **Age:** 18-100 years
     2. **Transaction Amount:** \$ 0-5000
     3. **Total Monthly Transactions:** \$ 0-50000
     4. **Annual Income:** \$ 30000-1000000
     5. **Gender:** 0/1 (0 - Male, 1 - Female)
     

-  **q2_labels.csv:** This file contains a 1000 × 1 vector of 0/1 labels for whether the transaction is fraudulent or not.
     <br><br>
    -  0: The transaction is legitimate
    -  1: The transaction is fraudulent
    
    
    
1. Report the confusion matrix and the F1 Score for this data set.
2. Which data set gives better results better? Can you think of reasons as to why one data set gives better results than the other? (**Hint**: Think of assumptions behind the logistic regression model)
3. Can you suggest improvements to the logistic regression model to make it perform better on the unfavorable data set?
4. **Bonus Points!**: Implement your suggested improvement as a code and compare the performance of this with vanilla logistic regression.

# Solution

### Load Data

In [22]:
features = pd.read_csv('../data/q2_data_matrix.csv', header=None, names=['age', 'tram', 'tomotr', 'anin', 'gen'])
labels = pd.read_csv('../data/q2_labels.csv', header=None, names=['fra'])